In [1]:
from collections import defaultdict
from collections import Counter
import json
import os
import math
import re
import string
import time
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from IPython.core.display import HTML
from tqdm import tqdm

transcripts = []
folders = ['app/data/debates/']
for folder in folders:
    for file_name in os.listdir(folder):
        with open(folder + file_name) as f:
            transcripts.append(json.load(f))

In [2]:
def tokenize(text):
    textlist = re.findall(r'[a-zA-Z]+',text)
    textlistfinal = []
    for x in textlist:
        textlistfinal.append(x.lower())
    return textlistfinal

flat_msgs = [x["text"] for transcript in transcripts for m in transcript["parts"] for x in m["text"]]
tokenized_list = []
full_transcript = []
for msg in flat_msgs:
    tokens = tokenize(msg)
    tokenized_list.append(tokens)
    full_transcript.extend(tokens)

In [3]:
dic = {}
for word in full_transcript: 
    if word in dic: 
        dic[word] += 1
    else: 
        dic[word] = 1
pruned_dict = {}
for word in dic:
    if dic[word] < 800 and dic[word] > 15: 
        pruned_dict[word] = dic[word]

In [4]:
co_occurence = np.zeros((len(pruned_dict), len(pruned_dict)))

for i, x in enumerate(pruned_dict): 
    for j, y in enumerate(pruned_dict): 
        for l in tokenized_list: 
            if x in l and y in l: 
                co_occurence[i,j] += 1
        co_occurence[i,j] = co_occurence[i,j] / pruned_dict[y]

KeyboardInterrupt: 

In [5]:
word_to_index = {}
index_to_word = {}
for i, word in enumerate(pruned_dict): 
    word_to_index[word] = i
    index_to_word[i] = word

co_occurence = np.zeros((len(pruned_dict), len(pruned_dict)))

for i in tqdm(range(len(pruned_dict))): 
    for j in range(i): 
        for l in tokenized_list: 
            if index_to_word[i] in l and index_to_word[j] in l: 
                co_occurence[i,j] += 1
                co_occurence[j,i] += 1

100%|██████████| 1514/1514 [1:39:53<00:00,  3.96s/it]


In [10]:
for x in range(len(pruned_dict)): 
    co_occurence[:,x] = co_occurence[:,x] / pruned_dict[index_to_word[x]]


In [7]:
import pickle 

pickle.dump(co_occurence, open("matrix.pickle", "wb"))


In [9]:
new_pickle = pickle.load(open("matrix.pickle", "rb"))
print(new_pickle)

[[0.         0.35483871 0.03636364 ... 0.         0.         0.0625    ]
 [0.12021858 0.         0.01298701 ... 0.         0.         0.0625    ]
 [0.07650273 0.08064516 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00546448 0.01612903 0.         ... 0.         0.         0.        ]]


In [11]:
dictionary = {}
for word in pruned_dict: 
    args = np.argsort(co_occurence[word_to_index[word], :])[::-1]
    dictionary[word] = [index_to_word[arg] for arg in args[0:10]]

In [12]:
print(dictionary)

{'candidates': ['evening', 'joined', 'audience', 'super', 'chris', 'tuesday', 'statements', 'looks', 'final', 'campaigns'], 'welcome': ['statements', 'evening', 'joined', 'georgia', 'tuesday', 'las', 'gentlemen', 'final', 'vegas', 'limits'], 'vice': ['oppose', 'unlike', 'actions', 'structural', 'false', 'calls', 'jersey', 'lifetime', 'enemies', 'virus'], 'biden': ['oppose', 'structural', 'rates', 'calls', 'beating', 'presidents', 'unlike', 'employer', 'post', 'deported'], 'first': ['known', 'puts', 'joined', 'arms', 'levels', 'enemies', 'lastly', 'clearly', 'repeal', 'nations'], 'question': ['repeal', 'shootings', 'kurds', 'mandatory', 'playing', 'diverse', 'filibuster', 'final', 'deep', 'november'], 'last': ['insulin', 'code', 'rise', 'looked', 'somehow', 'forget', 'despite', 'profit', 'massive', 'unprecedented'], 'few': ['parent', 'page', 'quality', 'hate', 'reach', 'houston', 'core', 'david', 'decent', 'stake'], 'days': ['mandatory', 'individual', 'insulin', 'list', 'himself', 'even

In [13]:
with open('data.json', 'w') as outfile:
    json.dump(dictionary, outfile)

In [15]:
print(pruned_dict["climate"])

267
